In [1]:
%%capture
%load_ext autoreload
%autoreload 2
from setup_nb_env import *

In [2]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### <font color=blue> Some data statistics.

In [9]:
from epsampling.utils import load_latest_csv

data_dir = '/work/users/k/4/k4thryn/Repos/EpSampling/data'

_df,name = load_latest_csv('merged_covs_deaths',f'{data_dir}/processed/')
_df.drop('Postal',axis=1,inplace=True)
_df.rename({'COVIDhubEns_state_deaths':'State_deaths_ens','True_county_deaths':'Deaths_true',
           'Naive_county_deaths':'Deaths_naive'},axis=1,inplace=True)
display(_df)

print('\n','Num samples:',len(_df),'\n',
      'Num dates:',_df.Date.nunique(),'\n',
      'Num counties:',_df.Fips.nunique(),'\n')

# acs_covs = df.columns[10:]
# death_covs = df.columns[:10]

# df_acs = df[acs_covs]
# df_deaths = df[death_covs]

,State_fips,State,County,Fips,Date,State_deaths_ens,Pop,Pop_ratio,Deaths_true,Deaths_naive,...,HU_x15,HU_UIS01D,HU_UIS01A,HU_UIS02,HU_UIS0304,HU_UIS0509,HU_UIS1019,HU_UIS2049,HU_UIS50P,HU_UISOTHER
0,41,OR,Baker,41001,2020-05-09,125.655,16124,0.004,0.000,0.480,...,8654,6300,109,162,159,183,105,90,157,1389
1,41,OR,Baker,41001,2020-05-16,143.950,16124,0.004,0.000,0.550,...,8654,6300,109,162,159,183,105,90,157,1389
2,41,OR,Baker,41001,2020-05-23,150.378,16124,0.004,0.000,0.575,...,8654,6300,109,162,159,183,105,90,157,1389
3,41,OR,Baker,41001,2020-05-30,159.093,16124,0.004,0.000,0.608,...,8654,6300,109,162,159,183,105,90,157,1389
4,41,OR,Baker,41001,2020-06-06,164.199,16124,0.004,0.000,0.628,...,8654,6300,109,162,159,183,105,90,157,1389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332432,24,MD,Baltimore city,24510,2022-04-09,14367.000,593490,0.098,1765.000,1410.374,...,293718,43042,149324,14615,16871,17300,12674,7999,31436,457
332433,24,MD,Baltimore city,24510,2022-04-16,14391.000,593490,0.098,1771.000,1412.730,...,293718,43042,149324,14615,16871,17300,12674,7999,31436,457
332434,24,MD,Baltimore city,24510,2022-04-23,14423.000,593490,0.098,1771.000,1415.872,...,293718,43042,149324,14615,16871,17300,12674,7999,31436,457
332435,24,MD,Baltimore city,24510,2022-04-30,14443.000,593490,0.098,1773.000,1417.835,...,293718,43042,149324,14615,16871,17300,12674,7999,31436,457



 Num samples: 332437 
 Num dates: 107 
 Num counties: 3130 



### <font color=blue> Do naive metrics vary a lot among counties?

In [10]:
dff = _df[['State_fips', 'State', 'County', 'Fips', 'Date', 'State_deaths_ens',
       'Pop', 'Pop_ratio', 'Deaths_true', 'Deaths_naive']]
dff

,State_fips,State,County,Fips,Date,State_deaths_ens,Pop,Pop_ratio,Deaths_true,Deaths_naive
0,41,OR,Baker,41001,2020-05-09,125.655,16124,0.004,0.000,0.480
1,41,OR,Baker,41001,2020-05-16,143.950,16124,0.004,0.000,0.550
2,41,OR,Baker,41001,2020-05-23,150.378,16124,0.004,0.000,0.575
3,41,OR,Baker,41001,2020-05-30,159.093,16124,0.004,0.000,0.608
4,41,OR,Baker,41001,2020-06-06,164.199,16124,0.004,0.000,0.628
...,...,...,...,...,...,...,...,...,...,...
332432,24,MD,Baltimore city,24510,2022-04-09,14367.000,593490,0.098,1765.000,1410.374
332433,24,MD,Baltimore city,24510,2022-04-16,14391.000,593490,0.098,1771.000,1412.730
332434,24,MD,Baltimore city,24510,2022-04-23,14423.000,593490,0.098,1771.000,1415.872
332435,24,MD,Baltimore city,24510,2022-04-30,14443.000,593490,0.098,1773.000,1417.835


In [12]:
from epsampling.utils import get_performance
import pprint

county_res_dict = {}

for fips in dff.Fips.unique():
    df_fips = dff[dff.Fips==fips]
    
    y_naive = df_fips['Deaths_naive'].values
    y_test = df_fips['Deaths_true'].values

    model_names = ['Deaths_naive'] #,'Deaths_true']
    model_preds = [y_naive]
    
    metrics_dict = get_performance(model_names, model_preds, y_test, y_naive)
    county_res_dict[fips] = list(metrics_dict.values())[0]
    
df_res = pd.DataFrame([[county,metric,value] 
                       for county,d in county_res_dict.items() 
                       for metric,value in d.items()], 
                       columns = ['county','metric','value'])

df = df_res.pivot(index='county',columns='metric',values='value')
df

metric,MAE,MSE,r2,relMAE
county,,,,
1001,13.563,280.602,0.933,1.000
1003,128.481,20883.311,0.612,1.000
1005,2.901,14.041,0.987,1.000
1007,11.664,184.186,0.854,1.000
1009,7.296,74.700,0.988,1.000
...,...,...,...,...
56037,7.185,80.640,0.958,1.000
56039,22.096,847.469,-25.826,1.000
56041,10.802,183.194,-0.080,1.000


In [13]:
df.reset_index(inplace=True)
df.columns.name = None
df.index.name = None
df

,county,MAE,MSE,r2,relMAE
0,1001,13.563,280.602,0.933,1.000
1,1003,128.481,20883.311,0.612,1.000
2,1005,2.901,14.041,0.987,1.000
3,1007,11.664,184.186,0.854,1.000
4,1009,7.296,74.700,0.988,1.000
...,...,...,...,...,...
3125,56037,7.185,80.640,0.958,1.000
3126,56039,22.096,847.469,-25.826,1.000
3127,56041,10.802,183.194,-0.080,1.000
3128,56043,12.707,199.986,-0.045,1.000


In [ ]:
from epsampling.utils import get_point_pred_performance

y_naive = df['Deaths_naive'].values
y_test = df['Deaths_true'].values

model_names = ['Deaths_naive','Deaths_true']
model_preds = y_naive, y_test

import pprint
pprint.pp(get_point_pred_performance(model_names, model_preds, y_test, y_naive))

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sktime.performance_metrics.forecasting import mean_relative_absolute_error

In [ ]:
# rel_error = abs(y_test - y_naive)

df_res = pd.DataFrame()
df_res['y_naive'] = df['Deaths_naive'].values
df_res['y_test'] = df['Deaths_true'].values

In [ ]:
# squared diff
sq_error = (y_test - y_naive)**2
# absolute diff
abs_error = abs(y_test - y_naive)
#r2
y_mean = np.mean(y_test)
r2_error = (y_test - y_naive)**2 / (y_test - y_mean)**2

df_res['sq_error'] = sq_error
df_res['abs_error'] = abs_error
df_res['r2_error'] = r2_error

In [ ]:
1 - df_res['r2_error'].mean()

In [ ]:
df_res

In [ ]:
# display(df.Sq_error.describe(),
# df.Abs_error.describe(),
# df.r2_error.describe())

# display(df.Sq_error.var(),
# df.Abs_error.var(),
# df.r2_error.var())

In [ ]:
df.r2_error.values

In [ ]:

df.r2_error.max(), df.r2_error.min()